<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

# Chapter 7: Modeling Readiness

**Purpose:** Validate that data is ready for machine learning and identify potential pitfalls.

**What you'll learn:**
- How to assess if your data is ready for modeling
- How to detect potential data leakage before it ruins your model
- How to handle class imbalance in binary classification
- What preprocessing steps are needed before training

**Outputs:**
- Pre-modeling validation checklist with pass/fail status
- Detailed leakage risk assessment (including temporal leakage)
- Class imbalance analysis with strategy recommendations
- Overall readiness score

---

## Why Modeling Readiness Matters

| Issue | Impact | Detection |
|-------|--------|-----------|
| **Data Leakage** | Overly optimistic results that fail in production | High correlation with target, future information |
| **Class Imbalance** | Model ignores minority class | Ratio > 3:1 between classes |
| **Missing Values** | Model failures or bias | >50% missing in any column |
| **Insufficient Data** | Overfitting, poor generalization | <100 rows for simple models |

## 7.1 Setup

In [1]:
from customer_retention.analysis.auto_explorer import ExplorationFindings
from customer_retention.analysis.visualization import ChartBuilder, display_figure, display_table
from customer_retention.core.config.column_config import ColumnType
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
# === CONFIGURATION ===
from pathlib import Path

FINDINGS_DIR = Path("../experiments/findings")

findings_files = [f for f in FINDINGS_DIR.glob("*_findings.yaml") if "multi_dataset" not in f.name]
if not findings_files:
    raise FileNotFoundError(f"No findings files found in {FINDINGS_DIR}. Run notebook 01 first.")

findings_files.sort(key=lambda f: f.stat().st_mtime, reverse=True)
FINDINGS_PATH = str(findings_files[0])

print(f"Found {len(findings_files)} findings file(s)")
print(f"Using: {FINDINGS_PATH}")

findings = ExplorationFindings.load(FINDINGS_PATH)

# Load data with snapshot preference (uses temporal snapshots if available)
from customer_retention.stages.temporal import load_data_with_snapshot_preference, TEMPORAL_METADATA_COLS
df, data_source = load_data_with_snapshot_preference(findings, output_dir="../experiments/findings")
charts = ChartBuilder()

print(f"\nLoaded {len(df):,} rows from: {data_source}")

FileNotFoundError: No findings files found in ../experiments/findings. Run notebook 01 first.

## 7.2 Modeling Readiness Checklist

In [ ]:
checklist = []

has_target = findings.target_column is not None
checklist.append({"Check": "Target column identified", "Status": "Pass" if has_target else "Fail"})

has_features = len([c for c in findings.columns.values() 
                   if c.inferred_type not in [ColumnType.IDENTIFIER, ColumnType.TARGET]]) > 0
checklist.append({"Check": "Feature columns available", "Status": "Pass" if has_features else "Fail"})

high_missing = any(c.universal_metrics.get("null_percentage", 0) > 50 
                   for c in findings.columns.values())
checklist.append({"Check": "No columns with >50% missing", "Status": "Fail" if high_missing else "Pass"})

good_quality = findings.overall_quality_score >= 70
checklist.append({"Check": "Quality score >= 70", "Status": "Pass" if good_quality else "Warn"})

sufficient_rows = findings.row_count >= 100
checklist.append({"Check": "Sufficient sample size (>=100)", "Status": "Pass" if sufficient_rows else "Fail"})

print("Modeling Readiness Checklist:")
print("="*50)
checklist_df = pd.DataFrame(checklist)
display(checklist_df)

## 7.3 Class Imbalance Analysis

**📖 Understanding Class Imbalance:**
- **Ratio < 3:1** - Mild imbalance, standard methods work
- **Ratio 3:1 to 10:1** - Moderate imbalance, use stratified sampling + class weights
- **Ratio > 10:1** - Severe imbalance, consider SMOTE, undersampling, or focal loss

**⚠️ Why It Matters:**
- Imbalanced data causes models to predict the majority class
- Accuracy is misleading (80% accuracy when 80% is majority = useless model)
- Use F1-score, Precision, Recall, and AUC instead of accuracy

In [ ]:
if findings.target_column:
    target = findings.target_column
    target_series = df[target]
    
    print("=" * 70)
    print("CLASS IMBALANCE ANALYSIS")
    print("=" * 70)
    
    print(f"\nTarget Column: {target}")
    print(f"Target Type: {findings.target_type}")
    print(f"Missing Values: {target_series.isnull().sum()}")
    
    if findings.target_type == "binary":
        value_counts = target_series.value_counts()
        majority_class = value_counts.idxmax()
        minority_class = value_counts.idxmin()
        majority_count = value_counts.max()
        minority_count = value_counts.min()
        imbalance_ratio = majority_count / minority_count
        
        print(f"\n📊 CLASS DISTRIBUTION:")
        print(f"  Majority Class ({majority_class}): {majority_count:,} ({majority_count/len(df)*100:.1f}%)")
        print(f"  Minority Class ({minority_class}): {minority_count:,} ({minority_count/len(df)*100:.1f}%)")
        print(f"  Imbalance Ratio: {imbalance_ratio:.2f}:1")
        
        # Determine severity and recommendations
        print(f"\n🎯 IMBALANCE ASSESSMENT:")
        if imbalance_ratio > 10:
            severity = "SEVERE"
            color = "#d62728"
            recommendations = [
                "Use SMOTE or ADASYN for oversampling",
                "Consider undersampling majority class",
                "Use focal loss for neural networks",
                "Use class_weight='balanced' in sklearn models"
            ]
        elif imbalance_ratio > 3:
            severity = "MODERATE"
            color = "#ffbb00"
            recommendations = [
                "Use stratified train/test split",
                "Use class_weight='balanced' in sklearn models",
                "Monitor Precision-Recall instead of just accuracy",
                "Consider threshold tuning"
            ]
        else:
            severity = "MILD"
            color = "#2ca02c"
            recommendations = [
                "Standard methods should work",
                "Use stratified cross-validation",
                "Monitor both classes in evaluation"
            ]
        
        print(f"  Severity: {severity}")
        print(f"\n📋 RECOMMENDATIONS:")
        for i, rec in enumerate(recommendations, 1):
            print(f"  {i}. {rec}")
        
        # Visualize
        fig = go.Figure(go.Bar(
            x=['Churned (0)', 'Retained (1)'],
            y=[value_counts.get(0, 0), value_counts.get(1, 0)],
            marker_color=['#d62728', '#2ca02c'],
            text=[f'{value_counts.get(0, 0):,}<br>({value_counts.get(0, 0)/len(df)*100:.1f}%)',
                  f'{value_counts.get(1, 0):,}<br>({value_counts.get(1, 0)/len(df)*100:.1f}%)'],
            textposition='outside'
        ))
        fig.update_layout(
            title=f'Target Class Distribution (Imbalance: {imbalance_ratio:.1f}:1 - {severity})',
            xaxis_title='Class',
            yaxis_title='Count',
            template='plotly_white',
            height=400
        )
        display_figure(fig)
        
        # Calculate what balanced class weights would be
        print(f"\n💡 CLASS WEIGHTS FOR SKLEARN:")
        weight_minority = len(df) / (2 * minority_count)
        weight_majority = len(df) / (2 * majority_count)
        print(f"  class_weight={{0: {weight_minority:.3f}, 1: {weight_majority:.3f}}}")
        print(f"  Or use class_weight='balanced'")
else:
    print("ERROR: No target column identified. Please set one in findings.")

## 7.4 Data Leakage Risk Assessment

**📖 Types of Leakage:**
- **Target Leakage**: Feature contains information about the target that wouldn't be available at prediction time
- **Train-Test Leakage**: Information from test set leaks into training (e.g., scaling before split)
- **Temporal Leakage**: Using future information to predict past events

**⚠️ Warning Signs:**
- Correlation > 0.9 with target (suspiciously predictive)
- Column names containing 'future', 'outcome', 'result'
- Date columns that come after the target determination date

In [ ]:
leakage_risks = []

if findings.target_column:
    target = findings.target_column
    
    for col_name, col_info in findings.columns.items():
        if col_name == target or col_info.inferred_type == ColumnType.IDENTIFIER:
            continue
        
        if col_info.inferred_type in [ColumnType.NUMERIC_CONTINUOUS, ColumnType.NUMERIC_DISCRETE]:
            corr = df[[col_name, target]].corr().iloc[0, 1]
            if abs(corr) > 0.9:
                leakage_risks.append({
                    "Column": col_name,
                    "Risk": "High",
                    "Reason": f"Very high correlation ({corr:.3f}) - potential leakage"
                })
        
        if any(kw in col_name.lower() for kw in ['future', 'outcome', 'result', 'after']):
            leakage_risks.append({
                "Column": col_name,
                "Risk": "Medium",
                "Reason": "Name suggests post-prediction information"
            })

if leakage_risks:
    print("Potential Leakage Risks:")
    display(pd.DataFrame(leakage_risks))
else:
    print("No obvious leakage risks detected.")

## 7.5 Feature Type Summary

In [ ]:
type_summary = {}
for col_info in findings.columns.values():
    col_type = col_info.inferred_type.value
    type_summary[col_type] = type_summary.get(col_type, 0) + 1

print("Feature Type Distribution:")
for col_type, count in sorted(type_summary.items()):
    print(f"  {col_type}: {count}")

usable_features = sum(1 for c in findings.columns.values() 
                      if c.inferred_type not in [ColumnType.IDENTIFIER, ColumnType.TARGET])
print(f"\nUsable features for modeling: {usable_features}")

## 7.6 Readiness Score

In [ ]:
scores = []

scores.append(25 if has_target else 0)
scores.append(25 if has_features else 0)
scores.append(25 if not high_missing else 10)
scores.append(25 if good_quality else 15)

readiness_score = sum(scores)

print(f"\nModeling Readiness Score: {readiness_score}/100")

if readiness_score >= 90:
    print("Status: READY - Proceed to modeling.")
elif readiness_score >= 70:
    print("Status: MOSTLY READY - Address minor issues before modeling.")
elif readiness_score >= 50:
    print("Status: NEEDS WORK - Significant issues to resolve.")
else:
    print("Status: NOT READY - Major issues must be fixed first.")

---

## Summary: What We Learned

In this notebook, we validated modeling readiness:

1. **Pre-modeling Checklist** - Verified target, features, missing values, and sample size
2. **Class Imbalance** - Analyzed distribution and provided mitigation strategies
3. **Leakage Assessment** - Checked for suspicious correlations and temporal issues
4. **Feature Summary** - Reviewed usable features by type

## Key Actions Before Modeling

| Action | Priority | Implementation |
|--------|----------|----------------|
| Use stratified splits | High | `train_test_split(..., stratify=y)` |
| Handle imbalance | High | `class_weight='balanced'` or SMOTE |
| Scale features | Medium | `StandardScaler` (fit on train only!) |
| Encode categoricals | Medium | One-hot or target encoding |

---

## Next Steps

Continue to **08_baseline_experiments.ipynb** to:
- Train baseline models with proper handling
- Compare model performance
- Analyze feature importance
- Evaluate with appropriate metrics (not just accuracy!)